In [1]:
'''
此代码目前仅可分析某一天（单文件）的某个时间段（预定的时间区间）的订单和位置数据
'''

import pandas as pd
import numpy as np
import time
import os
from utm import *
from tqdm import tqdm, tqdm_pandas
from osgeo import osr

In [2]:
import json
import urllib
import math

x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方


class Geocoding:
    def __init__(self, api_key):
        self.api_key = api_key

    def geocode(self, address):
        """
        利用高德geocoding服务解析地址获取位置坐标
        :param address:需要解析的地址
        :return:
        """
        geocoding = {'s': 'rsv3',
                     'key': self.api_key,
                     'city': '全国',
                     'address': address}
        geocoding = urllib.urlencode(geocoding)
        ret = urllib.urlopen("%s?%s" % ("http://restapi.amap.com/v3/geocode/geo", geocoding))

        if ret.getcode() == 200:
            res = ret.read()
            json_obj = json.loads(res)
            if json_obj['status'] == '1' and int(json_obj['count']) >= 1:
                geocodes = json_obj['geocodes'][0]
                lng = float(geocodes.get('location').split(',')[0])
                lat = float(geocodes.get('location').split(',')[1])
                return [lng, lat]
            else:
                return None
        else:
            return None


def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret


def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

In [3]:
# 在此处设置一些文件地址
feature_file_name = 'no mesh 10-31'
csv_path = 'F:/大学/第40期PRP/交通订单数据/traffic_data/gps_20161101.csv'
feature_dst_path = 'F:/大学/第40期PRP/特征提取/' + feature_file_name + '.csv'

In [4]:
# 在此处设置时间窗(秒)和空间网格的边长(WGS84坐标系)
time_interval = 600
space_interval = 70

In [5]:
# 设置时间区间 读取原数据
# 时间区间: 减少单次的处理量
time1 = '2016 11 01 08:00:00'
time2 = '2016 11 01 10:00:00'
stamp1 = time.mktime(time.strptime(time1, '%Y %m %d %H:%M:%S'))
stamp2 = time.mktime(time.strptime(time2, '%Y %m %d %H:%M:%S'))
#读取原地理数据
df = pd.read_csv(csv_path, header = None) #注意我此处使用的是移动硬盘的地址
df.columns = ['driver_ID', 'order_ID', 'timestamp', 'lon', 'lat']
df

,driver_ID,order_ID,timestamp,lon,lat
0,opqlvh8hc1yjyh6iCBooqrkhdmi_stBe,jjylseig5_zoyi_rrrndqzyndmd1zpwl,1477969147,104.07513,30.72724
1,opqlvh8hc1yjyh6iCBooqrkhdmi_stBe,jjylseig5_zoyi_rrrndqzyndmd1zpwl,1477969150,104.07513,30.72702
2,opqlvh8hc1yjyh6iCBooqrkhdmi_stBe,jjylseig5_zoyi_rrrndqzyndmd1zpwl,1477969154,104.07504,30.72672
3,opqlvh8hc1yjyh6iCBooqrkhdmi_stBe,jjylseig5_zoyi_rrrndqzyndmd1zpwl,1477969156,104.07497,30.72630
4,opqlvh8hc1yjyh6iCBooqrkhdmi_stBe,jjylseig5_zoyi_rrrndqzyndmd1zpwl,1477969159,104.07497,30.72582
...,...,...,...,...,...
32155512,vbuyneib6_soAf_ptoliuosnki4-yrBo,raClt9agh-ypqd4cosgaCuxfdj57Cvtb,1477976392,104.10102,30.67833
32155513,vbuyneib6_soAf_ptoliuosnki4-yrBo,raClt9agh-ypqd4cosgaCuxfdj57Cvtb,1477976393,104.10101,30.67833
32155514,vbuyneib6_soAf_ptoliuosnki4-yrBo,raClt9agh-ypqd4cosgaCuxfdj57Cvtb,1477976394,104.10100,30.67834
32155515,vbuyneib6_soAf_ptoliuosnki4-yrBo,raClt9agh-ypqd4cosgaCuxfdj57Cvtb,1477976396,104.10100,30.67834


In [6]:
# 空间坐标系转换
df = df[(df['timestamp'] >= stamp1)&(df['timestamp'] <= stamp2)&(df['timestamp']%6==1)].reset_index(drop = True)


In [7]:
df

,driver_ID,order_ID,timestamp,lon,lat
0,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,1477960657,104.12503,30.65755
1,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,1477960663,104.12420,30.65719
2,potvwmdihbvxqfamCqlksxljdb69tixp,hjqorfcif-tsng9gvtrjwrliaoa3yvpk,1477961497,104.09661,30.66272
3,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,1477962169,104.08389,30.65567
4,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,1477962175,104.08370,30.65576
...,...,...,...,...,...
619501,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961323,104.08969,30.66254
619502,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961329,104.08935,30.66270
619503,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961341,104.08919,30.66278
619504,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961377,104.08921,30.66277


In [8]:
# 定义坐标系转换

# 4.转换原数据的坐标
xy = df[['lon','lat']].apply(lambda x: gcj02_to_wgs84(x[0],x[1])[:2], axis = 1)


In [9]:
df

,driver_ID,order_ID,timestamp,lon,lat
0,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,1477960657,104.12503,30.65755
1,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,1477960663,104.12420,30.65719
2,potvwmdihbvxqfamCqlksxljdb69tixp,hjqorfcif-tsng9gvtrjwrliaoa3yvpk,1477961497,104.09661,30.66272
3,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,1477962169,104.08389,30.65567
4,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,1477962175,104.08370,30.65576
...,...,...,...,...,...
619501,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961323,104.08969,30.66254
619502,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961329,104.08935,30.66270
619503,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961341,104.08919,30.66278
619504,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961377,104.08921,30.66277


In [10]:
# 将所生成的xy坐标系数据写入dataframe
df1 = df
df1['lon'] = [x[0] for x in xy]
df1['lat'] = [x[1] for x in xy]
df1

,driver_ID,order_ID,timestamp,lon,lat
0,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,1477960657,104.122499,30.659963
1,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,1477960663,104.121668,30.659602
2,potvwmdihbvxqfamCqlksxljdb69tixp,hjqorfcif-tsng9gvtrjwrliaoa3yvpk,1477961497,104.094069,30.665116
3,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,1477962169,104.081357,30.658073
4,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,1477962175,104.081167,30.658164
...,...,...,...,...,...
619501,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961323,104.087152,30.664937
619502,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961329,104.086812,30.665098
619503,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961341,104.086652,30.665178
619504,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,1477961377,104.086672,30.665168


In [11]:
pd.to_datetime(df['timestamp'], unit = 's')

0        2016-11-01 00:37:37
1        2016-11-01 00:37:43
2        2016-11-01 00:51:37
3        2016-11-01 01:02:49
4        2016-11-01 01:02:55
                 ...        
619501   2016-11-01 00:48:43
619502   2016-11-01 00:48:49
619503   2016-11-01 00:49:01
619504   2016-11-01 00:49:37
619505   2016-11-01 00:56:49
Name: timestamp, Length: 619506, dtype: datetime64[ns]

In [12]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 's')

In [13]:
df

,driver_ID,order_ID,timestamp,lon,lat
0,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,2016-11-01 00:37:37,104.122499,30.659963
1,potvwmdihbvxqfamCqlksxljdb69tixp,lpqmkhaf9aHtooanvwsjzsjn8l95Eltk,2016-11-01 00:37:43,104.121668,30.659602
2,potvwmdihbvxqfamCqlksxljdb69tixp,hjqorfcif-tsng9gvtrjwrliaoa3yvpk,2016-11-01 00:51:37,104.094069,30.665116
3,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,2016-11-01 01:02:49,104.081357,30.658073
4,qjvorakea0Dtth5hzwqaCtl88a49Autc,jfBsmjhijaytpb4gpBkjzzsnjkg.wqyg,2016-11-01 01:02:55,104.081167,30.658164
...,...,...,...,...,...
619501,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,2016-11-01 00:48:43,104.087152,30.664937
619502,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,2016-11-01 00:48:49,104.086812,30.665098
619503,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,2016-11-01 00:49:01,104.086652,30.665178
619504,lhwAvh8ne2Csyd_qyrtoulmlbm6aCizd,kixotjimjaGqnc5oDviouAyfahfbEqvi,2016-11-01 00:49:37,104.086672,30.665168


In [14]:
# 导出数据
df.to_csv(feature_dst_path, index = None)